In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#!pip install --upgrade plotly
#!pip install pmdarima
print("Done")

Done


In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot, autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from math import sqrt
import itertools

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.ar_model import AR
from pmdarima import auto_arima

import warnings
warnings.filterwarnings("ignore")


In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestRegressor

SEED = 42

In [5]:
df = pd.read_csv("/content/gdrive/MyDrive/owid-covid-data.csv")

In [6]:
df.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_vaccinations_smoothed', 'total_vaccinations_per_hun

In [7]:
dataset_India = df[df["location"] == "India"]          
dataset_USA = df[df["location"] == "United States"]    
dataset_UK = df[df["location"] == "United Kingdom"]    
dataset_Canada = df[df["location"] == "Canada"]        
dataset_Italy = df[df["location"] == "Italy"]   

In [8]:
dataset_India.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
39476,IND,Asia,India,2020-01-30,1.0,1.0,NaN,NaN,NaN,NaN,0.001,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.19,1.380004e+09,450.419,28.2,5.989,3.414,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.645,NaN
39477,IND,Asia,India,2020-01-31,1.0,0.0,NaN,NaN,NaN,NaN,0.001,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.19,1.380004e+09,450.419,28.2,5.989,3.414,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.645,NaN
39478,IND,Asia,India,2020-02-01,1.0,0.0,NaN,NaN,NaN,NaN,0.001,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.19,1.380004e+09,450.419,28.2,5.989,3.414,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.645,NaN
39479,IND,Asia,India,2020-02-02,2.0,1.0,NaN,NaN,NaN,NaN,0.001,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.19,1.380004e+09,450.419,28.2,5.989,3.414,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.645,NaN
39480,IND,Asia,India,2020-02-03,3.0,1.0,NaN,NaN,NaN,NaN,0.002,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.19,1.380004e+09,450.419,28.2,5.989,3.414,6426.674,21.2,282.28,10.39,1.9,20.6,59.55,0.53,69.66,0.645,NaN


In [9]:
dataset_UK = dataset_UK[dataset_UK["date"] >= "2020-05-01"]
dataset_India = dataset_India[dataset_India["date"] >= "2020-05-01"]
dataset_Canada = dataset_Canada[dataset_Canada["date"] >= "2020-05-01"]
dataset_USA = dataset_USA[dataset_USA["date"] >= "2020-05-01"]
dataset_Italy = dataset_Italy[dataset_Italy["date"] >= "2020-05-01"]

In [10]:
dataset_UK["date"].head()

90078    2020-05-01
90079    2020-05-02
90080    2020-05-03
90081    2020-05-04
90082    2020-05-05
Name: date, dtype: object

In [11]:
dataset_UK["date"].tail()

90482    2021-06-09
90483    2021-06-10
90484    2021-06-11
90485    2021-06-12
90486    2021-06-13
Name: date, dtype: object

In [12]:
len(dataset_UK)

409

In [13]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.026,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [14]:
features = ["location", "date", "new_deaths_per_million", "new_cases_per_million", "icu_patients_per_million", "hosp_patients_per_million", "new_tests_per_thousand", "total_vaccinations_per_hundred", "gdp_per_capita", "population", "reproduction_rate", "people_fully_vaccinated_per_hundred", "people_vaccinated_per_hundred", "new_cases", "new_deaths"]

In [15]:
dataset_UK = dataset_UK[features]
dataset_India = dataset_India[features]
dataset_Italy = dataset_Italy[features]
dataset_USA = dataset_USA[features]
dataset_Canada = dataset_Canada[features]

In [16]:
dataset_UK.head()

,location,date,new_deaths_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,new_tests_per_thousand,total_vaccinations_per_hundred,gdp_per_capita,population,reproduction_rate,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred,new_cases,new_deaths
90078,United Kingdom,2020-05-01,10.311,69.676,32.083,216.878,1.092,NaN,39753.244,67886004.0,0.94,NaN,NaN,4730.0,700.0
90079,United Kingdom,2020-05-02,8.617,47.580,30.463,211.826,0.936,NaN,39753.244,67886004.0,0.90,NaN,NaN,3230.0,585.0
90080,United Kingdom,2020-05-03,3.727,43.853,29.594,208.997,1.005,NaN,39753.244,67886004.0,0.87,NaN,NaN,2977.0,253.0
90081,United Kingdom,2020-05-04,4.021,49.834,28.518,207.657,1.070,NaN,39753.244,67886004.0,0.82,NaN,NaN,3383.0,273.0
90082,United Kingdom,2020-05-05,10.680,54.341,27.605,202.148,0.986,NaN,39753.244,67886004.0,0.81,NaN,NaN,3689.0,725.0


In [17]:
dataset_India["Mortality_Rate"] = " "
dataset_Canada["Mortality_Rate"] = " "
dataset_UK["Mortality_Rate"] = " "
dataset_USA["Mortality_Rate"] = " "
dataset_Italy["Mortality_Rate"] = " "

In [18]:
dataset_India["Fatality_Rate"] = " "
dataset_Canada["Fatality_Rate"] = " "
dataset_UK["Fatality_Rate"] = " "
dataset_USA["Fatality_Rate"] = " "
dataset_Italy["Fatality_Rate"] = " "

In [19]:
dataset_India["Mortality_Rate"] = dataset_India["new_deaths_per_million"]/dataset_India["new_cases_per_million"]
dataset_USA["Mortality_Rate"] = dataset_USA["new_deaths_per_million"]/dataset_USA["new_cases_per_million"]
dataset_UK["Mortality_Rate"] = dataset_UK["new_deaths_per_million"]/dataset_UK["new_cases_per_million"]
dataset_Canada["Mortality_Rate"] = dataset_Canada["new_deaths_per_million"]/dataset_Canada["new_cases_per_million"]
dataset_Italy["Mortality_Rate"] = dataset_Italy["new_deaths_per_million"]/dataset_Italy["new_cases_per_million"]

In [20]:
dataset_India["Fatality_Rate"] = dataset_India["new_deaths"]/dataset_India["new_cases"]
dataset_USA["Fatality_Rate"] = dataset_USA["new_deaths"]/dataset_USA["new_cases"]
dataset_UK["Fatality_Rate"] = dataset_UK["new_deaths"]/dataset_UK["new_cases"]
dataset_Canada["Fatality_Rate"] = dataset_Canada["new_deaths"]/dataset_Canada["new_cases"]
dataset_Italy["Fatality_Rate"] = dataset_Italy["new_deaths"]/dataset_Italy["new_cases"]

In [21]:
dataset_India.head()

,location,date,new_deaths_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,new_tests_per_thousand,total_vaccinations_per_hundred,gdp_per_capita,population,reproduction_rate,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred,new_cases,new_deaths,Mortality_Rate,Fatality_Rate
39568,India,2020-05-01,0.050,1.735,NaN,NaN,0.053,NaN,6426.674,1.380004e+09,1.41,NaN,NaN,2394.0,69.0,0.028818,0.028822
39569,India,2020-05-02,0.072,1.770,NaN,NaN,0.053,NaN,6426.674,1.380004e+09,1.41,NaN,NaN,2442.0,100.0,0.040678,0.040950
39570,India,2020-05-03,0.049,2.033,NaN,NaN,0.051,NaN,6426.674,1.380004e+09,1.42,NaN,NaN,2806.0,68.0,0.024102,0.024234
39571,India,2020-05-04,0.127,2.849,NaN,NaN,0.044,NaN,6426.674,1.380004e+09,1.42,NaN,NaN,3932.0,175.0,0.044577,0.044507
39572,India,2020-05-05,0.092,2.147,NaN,NaN,0.061,NaN,6426.674,1.380004e+09,1.41,NaN,NaN,2963.0,127.0,0.042850,0.042862


In [22]:
dataset_India.isna().sum()

location                                 0
date                                     0
new_deaths_per_million                   0
new_cases_per_million                    0
icu_patients_per_million               409
hosp_patients_per_million              409
new_tests_per_thousand                  10
total_vaccinations_per_hundred         264
gdp_per_capita                           0
population                               0
reproduction_rate                        4
people_fully_vaccinated_per_hundred    293
people_vaccinated_per_hundred          264
new_cases                                0
new_deaths                               0
Mortality_Rate                           2
Fatality_Rate                            2
dtype: int64

In [23]:
dataset_UK.isna().sum()

location                                 0
date                                     0
new_deaths_per_million                   0
new_cases_per_million                    0
icu_patients_per_million                 3
hosp_patients_per_million                3
new_tests_per_thousand                   4
total_vaccinations_per_hundred         254
gdp_per_capita                           0
population                               0
reproduction_rate                        4
people_fully_vaccinated_per_hundred    254
people_vaccinated_per_hundred          254
new_cases                                0
new_deaths                               0
Mortality_Rate                           0
Fatality_Rate                            0
dtype: int64

In [24]:
dataset_USA.isna().sum()

location                                 0
date                                     0
new_deaths_per_million                   0
new_cases_per_million                    0
icu_patients_per_million                76
hosp_patients_per_million               76
new_tests_per_thousand                   5
total_vaccinations_per_hundred         248
gdp_per_capita                           0
population                               0
reproduction_rate                        4
people_fully_vaccinated_per_hundred    263
people_vaccinated_per_hundred          249
new_cases                                0
new_deaths                               0
Mortality_Rate                           0
Fatality_Rate                            0
dtype: int64

In [25]:
dataset_Italy.isna().sum()

location                                 0
date                                     0
new_deaths_per_million                   0
new_cases_per_million                    0
icu_patients_per_million                 7
hosp_patients_per_million                7
new_tests_per_thousand                   3
total_vaccinations_per_hundred         240
gdp_per_capita                           0
population                               0
reproduction_rate                        4
people_fully_vaccinated_per_hundred    261
people_vaccinated_per_hundred          240
new_cases                                0
new_deaths                               0
Mortality_Rate                           0
Fatality_Rate                            0
dtype: int64

In [26]:
dataset_Canada.isna().sum()

location                                 0
date                                     0
new_deaths_per_million                   0
new_cases_per_million                    0
icu_patients_per_million                 1
hosp_patients_per_million                1
new_tests_per_thousand                   3
total_vaccinations_per_hundred         227
gdp_per_capita                           0
population                               0
reproduction_rate                        4
people_fully_vaccinated_per_hundred    250
people_vaccinated_per_hundred          227
new_cases                                0
new_deaths                               0
Mortality_Rate                           0
Fatality_Rate                            0
dtype: int64

In [27]:
dataset_India.columns

Index(['location', 'date', 'new_deaths_per_million', 'new_cases_per_million',
       'icu_patients_per_million', 'hosp_patients_per_million',
       'new_tests_per_thousand', 'total_vaccinations_per_hundred',
       'gdp_per_capita', 'population', 'reproduction_rate',
       'people_fully_vaccinated_per_hundred', 'people_vaccinated_per_hundred',
       'new_cases', 'new_deaths', 'Mortality_Rate', 'Fatality_Rate'],
      dtype='object')

In [28]:
dataset_UK["people_fully_vaccinated_per_hundred"] = dataset_UK["people_fully_vaccinated_per_hundred"].fillna(0)
dataset_UK["people_vaccinated_per_hundred"] = dataset_UK["people_vaccinated_per_hundred"].fillna(0)
dataset_UK["new_tests_per_thousand"] = dataset_UK["new_tests_per_thousand"].fillna(0)
dataset_UK["hosp_patients_per_million"] = dataset_UK["hosp_patients_per_million"].fillna(0)   #This need to be deleted as not enough data is available.
dataset_UK["icu_patients_per_million"] = dataset_UK["icu_patients_per_million"].fillna(0)   #This need to be deleted as not enough data is available.

In [29]:
dataset_UK["date"].min()

'2020-05-01'

In [30]:
dataset_UK["Lockdown"] = ""

In [31]:
dataset_UK["date"].head()

90078    2020-05-01
90079    2020-05-02
90080    2020-05-03
90081    2020-05-04
90082    2020-05-05
Name: date, dtype: object

In [32]:
yes1 = (dataset_UK["date"] >= "2020-05-10") & (dataset_UK["date"] <= "2020-06-23")
yes2 = (dataset_UK["date"] >= "2020-10-31") & (dataset_UK["date"] <= "2020-12-02")
yes3 = (dataset_UK["date"] >= "2020-12-21") & (dataset_UK["date"] <= "2021-04-17")

In [33]:
dataset_UK["Lockdown"][yes1] = 1
dataset_UK["Lockdown"][yes2] = 1
dataset_UK["Lockdown"][yes3] = 1

In [34]:
dataset_UK["Lockdown"].unique()

array(['', 1], dtype=object)

In [35]:
dataset_UK[dataset_UK["Lockdown"] == 1]

,location,date,new_deaths_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,new_tests_per_thousand,total_vaccinations_per_hundred,gdp_per_capita,population,reproduction_rate,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred,new_cases,new_deaths,Mortality_Rate,Fatality_Rate,Lockdown
90087,United Kingdom,2020-05-10,3.197,34.204,22.656,172.745,1.127,NaN,39753.244,67886004.0,0.84,0.00,0.00,2322.0,217.0,0.093469,0.093454,1
90088,United Kingdom,2020-05-11,2.769,52.868,21.875,176.281,1.036,NaN,39753.244,67886004.0,0.85,0.00,0.00,3589.0,188.0,0.052376,0.052382,1
90089,United Kingdom,2020-05-12,9.059,50.010,20.829,170.742,1.099,NaN,39753.244,67886004.0,0.84,0.00,0.00,3395.0,615.0,0.181144,0.181149,1
90090,United Kingdom,2020-05-13,6.599,48.611,19.768,167.074,1.225,NaN,39753.244,67886004.0,0.81,0.00,0.00,3300.0,448.0,0.135751,0.135758,1
90091,United Kingdom,2020-05-14,5.200,38.565,19.356,160.873,1.248,NaN,39753.244,67886004.0,0.79,0.00,0.00,2618.0,353.0,0.134837,0.134836,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90425,United Kingdom,2021-04-13,0.339,36.900,5.450,35.456,10.004,59.65,39753.244,67886004.0,0.91,12.04,47.62,2505.0,23.0,0.009187,0.009182,1
90426,United Kingdom,2021-04-14,0.560,37.254,5.170,33.792,17.129,60.33,39753.244,67886004.0,0.90,12.54,47.79,2529.0,38.0,0.015032,0.015026,1
90427,United Kingdom,2021-04-15,0.457,40.745,4.846,32.319,14.558,61.14,39753.244,67886004.0,0.89,13.16,47.98,2766.0,31.0,0.011216,0.011208,1
90428,United Kingdom,2021-04-16,0.501,41.349,4.891,31.509,10.045,62.08,39753.244,67886004.0,0.90,13.89,48.18,2807.0,34.0,0.012116,0.012113,1


In [36]:
dataset_UK["Lockdown"].unique()

array(['', 1], dtype=object)

In [37]:
dataset_UK["Lockdown"] = dataset_UK["Lockdown"].replace([''], 0)

In [38]:
dataset_UK["Lockdown"].unique()

array([0, 1])

In [39]:
dataset_UK.columns

Index(['location', 'date', 'new_deaths_per_million', 'new_cases_per_million',
       'icu_patients_per_million', 'hosp_patients_per_million',
       'new_tests_per_thousand', 'total_vaccinations_per_hundred',
       'gdp_per_capita', 'population', 'reproduction_rate',
       'people_fully_vaccinated_per_hundred', 'people_vaccinated_per_hundred',
       'new_cases', 'new_deaths', 'Mortality_Rate', 'Fatality_Rate',
       'Lockdown'],
      dtype='object')

In [40]:
dataset_UK[dataset_UK["total_vaccinations_per_hundred"].isna()]

,location,date,new_deaths_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,new_tests_per_thousand,total_vaccinations_per_hundred,gdp_per_capita,population,reproduction_rate,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred,new_cases,new_deaths,Mortality_Rate,Fatality_Rate,Lockdown
90078,United Kingdom,2020-05-01,10.311,69.676,32.083,216.878,1.092,NaN,39753.244,67886004.0,0.94,0.0,0.0,4730.0,700.0,0.147985,0.147992,0
90079,United Kingdom,2020-05-02,8.617,47.580,30.463,211.826,0.936,NaN,39753.244,67886004.0,0.90,0.0,0.0,3230.0,585.0,0.181106,0.181115,0
90080,United Kingdom,2020-05-03,3.727,43.853,29.594,208.997,1.005,NaN,39753.244,67886004.0,0.87,0.0,0.0,2977.0,253.0,0.084988,0.084985,0
90081,United Kingdom,2020-05-04,4.021,49.834,28.518,207.657,1.070,NaN,39753.244,67886004.0,0.82,0.0,0.0,3383.0,273.0,0.080688,0.080698,0
90082,United Kingdom,2020-05-05,10.680,54.341,27.605,202.148,0.986,NaN,39753.244,67886004.0,0.81,0.0,0.0,3689.0,725.0,0.196537,0.196530,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90328,United Kingdom,2021-01-06,15.349,921.486,41.555,470.524,8.211,NaN,39753.244,67886004.0,1.19,0.0,0.0,62556.0,1042.0,0.016657,0.016657,1
90329,United Kingdom,2021-01-07,17.117,777.583,43.146,479.907,9.132,NaN,39753.244,67886004.0,1.14,0.0,0.0,52787.0,1162.0,0.022013,0.022013,1
90330,United Kingdom,2021-01-08,19.636,1004.507,45.635,497.511,10.020,NaN,39753.244,67886004.0,1.12,0.0,0.0,68192.0,1333.0,0.019548,0.019548,1
90331,United Kingdom,2021-01-09,15.246,885.278,46.328,501.252,8.357,NaN,39753.244,67886004.0,1.09,0.0,0.0,60098.0,1035.0,0.017222,0.017222,1


In [41]:
len(dataset_UK)

409

In [42]:
dataset_UK.dropna(subset = ["reproduction_rate"], axis = 0, inplace = True)

In [43]:
len(dataset_UK)

405

In [44]:
dataset_UK["reproduction_rate"].isna().sum()

0

In [45]:
dataset_UK.isna().sum()

location                                 0
date                                     0
new_deaths_per_million                   0
new_cases_per_million                    0
icu_patients_per_million                 0
hosp_patients_per_million                0
new_tests_per_thousand                   0
total_vaccinations_per_hundred         253
gdp_per_capita                           0
population                               0
reproduction_rate                        0
people_fully_vaccinated_per_hundred      0
people_vaccinated_per_hundred            0
new_cases                                0
new_deaths                               0
Mortality_Rate                           0
Fatality_Rate                            0
Lockdown                                 0
dtype: int64

In [46]:
dataset_UK["total_vaccinations_per_hundred"] = dataset_UK["total_vaccinations_per_hundred"].fillna(0)

In [47]:
dataset_UK.isna().sum()

location                               0
date                                   0
new_deaths_per_million                 0
new_cases_per_million                  0
icu_patients_per_million               0
hosp_patients_per_million              0
new_tests_per_thousand                 0
total_vaccinations_per_hundred         0
gdp_per_capita                         0
population                             0
reproduction_rate                      0
people_fully_vaccinated_per_hundred    0
people_vaccinated_per_hundred          0
new_cases                              0
new_deaths                             0
Mortality_Rate                         0
Fatality_Rate                          0
Lockdown                               0
dtype: int64

In [48]:
len(dataset_UK)

405

In [49]:
dataset_UK["Variant"] = ""
dataset_UK["Vaccine"] = ""

In [50]:
dataset_UK.columns

Index(['location', 'date', 'new_deaths_per_million', 'new_cases_per_million',
       'icu_patients_per_million', 'hosp_patients_per_million',
       'new_tests_per_thousand', 'total_vaccinations_per_hundred',
       'gdp_per_capita', 'population', 'reproduction_rate',
       'people_fully_vaccinated_per_hundred', 'people_vaccinated_per_hundred',
       'new_cases', 'new_deaths', 'Mortality_Rate', 'Fatality_Rate',
       'Lockdown', 'Variant', 'Vaccine'],
      dtype='object')

In [51]:
dataset_UK

,location,date,new_deaths_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,new_tests_per_thousand,total_vaccinations_per_hundred,gdp_per_capita,population,reproduction_rate,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred,new_cases,new_deaths,Mortality_Rate,Fatality_Rate,Lockdown,Variant,Vaccine
90078,United Kingdom,2020-05-01,10.311,69.676,32.083,216.878,1.092,0.00,39753.244,67886004.0,0.94,0.00,0.00,4730.0,700.0,0.147985,0.147992,0,,
90079,United Kingdom,2020-05-02,8.617,47.580,30.463,211.826,0.936,0.00,39753.244,67886004.0,0.90,0.00,0.00,3230.0,585.0,0.181106,0.181115,0,,
90080,United Kingdom,2020-05-03,3.727,43.853,29.594,208.997,1.005,0.00,39753.244,67886004.0,0.87,0.00,0.00,2977.0,253.0,0.084988,0.084985,0,,
90081,United Kingdom,2020-05-04,4.021,49.834,28.518,207.657,1.070,0.00,39753.244,67886004.0,0.82,0.00,0.00,3383.0,273.0,0.080688,0.080698,0,,
90082,United Kingdom,2020-05-05,10.680,54.341,27.605,202.148,0.986,0.00,39753.244,67886004.0,0.81,0.00,0.00,3689.0,725.0,0.196537,0.196530,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90478,United Kingdom,2021-06-05,0.191,83.301,1.989,13.758,7.201,100.16,39753.244,67886004.0,1.48,40.75,59.41,5655.0,13.0,0.002293,0.002299,0,,
90479,United Kingdom,2021-06-06,0.059,76.967,2.136,14.141,18.296,100.73,39753.244,67886004.0,1.48,41.13,59.60,5225.0,4.0,0.000767,0.000766,0,,
90480,United Kingdom,2021-06-07,0.015,82.447,2.180,15.172,13.676,101.35,39753.244,67886004.0,1.48,41.58,59.77,5597.0,1.0,0.000182,0.000179,0,,
90481,United Kingdom,2021-06-08,0.206,87.986,2.269,15.482,10.713,102.01,39753.244,67886004.0,1.48,42.04,59.97,5973.0,14.0,0.002341,0.002344,0,,


In [52]:
alpha = (dataset_UK["date"] >= "2020-09-15") & (dataset_UK["date"] <= "2021-05-13")
delta = (dataset_UK["date"] > "2021-05-13") & (dataset_UK["date"] <= "2021-06-09")

In [53]:
dataset_UK["Variant"][alpha] = "Alpha"
dataset_UK["Variant"][delta] = "Delta"

In [54]:
dataset_UK["Variant"].unique()

array(['', 'Alpha', 'Delta'], dtype=object)

In [55]:
dataset_UK["Variant"] = dataset_UK["Variant"].replace([''], "First")

In [56]:
dataset_UK["Variant"].unique()

array(['First', 'Alpha', 'Delta'], dtype=object)

In [57]:
dataset_UK[dataset_UK["total_vaccinations_per_hundred"] == 0]

,location,date,new_deaths_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,new_tests_per_thousand,total_vaccinations_per_hundred,gdp_per_capita,population,reproduction_rate,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred,new_cases,new_deaths,Mortality_Rate,Fatality_Rate,Lockdown,Variant,Vaccine
90078,United Kingdom,2020-05-01,10.311,69.676,32.083,216.878,1.092,0.0,39753.244,67886004.0,0.94,0.0,0.0,4730.0,700.0,0.147985,0.147992,0,First,
90079,United Kingdom,2020-05-02,8.617,47.580,30.463,211.826,0.936,0.0,39753.244,67886004.0,0.90,0.0,0.0,3230.0,585.0,0.181106,0.181115,0,First,
90080,United Kingdom,2020-05-03,3.727,43.853,29.594,208.997,1.005,0.0,39753.244,67886004.0,0.87,0.0,0.0,2977.0,253.0,0.084988,0.084985,0,First,
90081,United Kingdom,2020-05-04,4.021,49.834,28.518,207.657,1.070,0.0,39753.244,67886004.0,0.82,0.0,0.0,3383.0,273.0,0.080688,0.080698,0,First,
90082,United Kingdom,2020-05-05,10.680,54.341,27.605,202.148,0.986,0.0,39753.244,67886004.0,0.81,0.0,0.0,3689.0,725.0,0.196537,0.196530,0,First,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90327,United Kingdom,2021-01-05,12.978,899.847,38.962,451.964,7.345,0.0,39753.244,67886004.0,1.24,0.0,0.0,61087.0,881.0,0.014422,0.014422,1,Alpha,
90328,United Kingdom,2021-01-06,15.349,921.486,41.555,470.524,8.211,0.0,39753.244,67886004.0,1.19,0.0,0.0,62556.0,1042.0,0.016657,0.016657,1,Alpha,
90329,United Kingdom,2021-01-07,17.117,777.583,43.146,479.907,9.132,0.0,39753.244,67886004.0,1.14,0.0,0.0,52787.0,1162.0,0.022013,0.022013,1,Alpha,
90330,United Kingdom,2021-01-08,19.636,1004.507,45.635,497.511,10.020,0.0,39753.244,67886004.0,1.12,0.0,0.0,68192.0,1333.0,0.019548,0.019548,1,Alpha,


In [58]:
no_vaccine = (dataset_UK["date"] >= "2020-05-01") & (dataset_UK["date"] <= "2021-01-09")
astra_zeneca = (dataset_UK["date"] > "2021-01-09") & (dataset_UK["date"] <= "2021-06-09")

In [59]:
dataset_UK["Vaccine"][no_vaccine] = "No Vaccine"
dataset_UK["Vaccine"][astra_zeneca] = "Astra Zeneca"

In [60]:
dataset_UK["Vaccine"].unique()

array(['No Vaccine', 'Astra Zeneca'], dtype=object)

In [61]:
dataset_UK[dataset_UK["Vaccine"] == "Astra Zeneca"]

,location,date,new_deaths_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,new_tests_per_thousand,total_vaccinations_per_hundred,gdp_per_capita,population,reproduction_rate,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred,new_cases,new_deaths,Mortality_Rate,Fatality_Rate,Lockdown,Variant,Vaccine
90332,United Kingdom,2021-01-10,8.352,810.565,48.081,521.271,7.157,3.94,39753.244,67886004.0,1.04,0.58,3.37,55026.0,567.0,0.010304,0.010304,1,Alpha,Astra Zeneca
90333,United Kingdom,2021-01-11,7.792,681.657,49.539,541.319,7.910,4.19,39753.244,67886004.0,0.98,0.61,3.58,46275.0,529.0,0.011431,0.011432,1,Alpha,Astra Zeneca
90334,United Kingdom,2021-01-12,18.354,672.421,51.498,544.693,8.614,4.52,39753.244,67886004.0,0.94,0.63,3.89,45648.0,1246.0,0.027295,0.027296,1,Alpha,Astra Zeneca
90335,United Kingdom,2021-01-13,23.098,702.089,53.413,552.146,9.259,4.94,39753.244,67886004.0,0.91,0.65,4.30,47662.0,1568.0,0.032899,0.032898,1,Alpha,Astra Zeneca
90336,United Kingdom,2021-01-14,18.457,718.911,54.091,555.137,10.240,5.42,39753.244,67886004.0,0.90,0.65,4.77,48804.0,1253.0,0.025674,0.025674,1,Alpha,Astra Zeneca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90478,United Kingdom,2021-06-05,0.191,83.301,1.989,13.758,7.201,100.16,39753.244,67886004.0,1.48,40.75,59.41,5655.0,13.0,0.002293,0.002299,0,Delta,Astra Zeneca
90479,United Kingdom,2021-06-06,0.059,76.967,2.136,14.141,18.296,100.73,39753.244,67886004.0,1.48,41.13,59.60,5225.0,4.0,0.000767,0.000766,0,Delta,Astra Zeneca
90480,United Kingdom,2021-06-07,0.015,82.447,2.180,15.172,13.676,101.35,39753.244,67886004.0,1.48,41.58,59.77,5597.0,1.0,0.000182,0.000179,0,Delta,Astra Zeneca
90481,United Kingdom,2021-06-08,0.206,87.986,2.269,15.482,10.713,102.01,39753.244,67886004.0,1.48,42.04,59.97,5973.0,14.0,0.002341,0.002344,0,Delta,Astra Zeneca


In [62]:
dataset_UK.isna().sum()

location                               0
date                                   0
new_deaths_per_million                 0
new_cases_per_million                  0
icu_patients_per_million               0
hosp_patients_per_million              0
new_tests_per_thousand                 0
total_vaccinations_per_hundred         0
gdp_per_capita                         0
population                             0
reproduction_rate                      0
people_fully_vaccinated_per_hundred    0
people_vaccinated_per_hundred          0
new_cases                              0
new_deaths                             0
Mortality_Rate                         0
Fatality_Rate                          0
Lockdown                               0
Variant                                0
Vaccine                                0
dtype: int64

In [63]:
fig = px.line(dataset_UK,  dataset_UK["date"], dataset_UK["new_deaths_per_million"], title = "New Deaths", color = dataset_UK["Variant"]) #hover_data = ["new_deaths", "population", "total_cases", "new_cases"])
fig.update_traces(mode = "markers + lines", hovertemplate = None)
fig.update_layout(hovermode = "x unified")
fig.show()

In [64]:
fig = px.line(dataset_UK,  dataset_UK["date"], dataset_UK["new_deaths_per_million"], title = "New Deaths", color = dataset_UK["Vaccine"]) #hover_data = ["new_deaths", "population", "total_cases", "new_cases"])
fig.update_traces(mode = "markers + lines", hovertemplate = None)
fig.update_layout(hovermode = "x unified")
fig.show()

In [65]:
fig = px.line(dataset_UK,  dataset_UK["date"], dataset_UK["new_deaths_per_million"], title = "New Deaths", color = dataset_UK["Vaccine"]) #hover_data = ["new_deaths", "population", "total_cases", "new_cases"])
fig.update_traces(mode = "markers + lines", hovertemplate = None)
fig.update_layout(hovermode = "x unified")
fig.show()

In [66]:
fig = px.line(dataset_UK,  dataset_UK["date"], dataset_UK["new_deaths"], title = "New Deaths", color = dataset_UK["Variant"]) #hover_data = ["new_deaths", "population", "total_cases", "new_cases"])
fig.update_traces(mode = "markers + lines", hovertemplate = None)
fig.update_layout(hovermode = "x unified")
fig.show()

In [67]:
fig = px.line(dataset_UK,  dataset_UK["date"], dataset_UK["new_cases"], title = "Mortality Rate", color = dataset_UK["Variant"]) #hover_data = ["new_deaths", "population", "total_cases", "new_cases"])
fig.update_traces(mode = "markers + lines", hovertemplate = None)
fig.update_layout(hovermode = "x unified")
fig.show()

In [68]:
fig = px.line(dataset_UK,  dataset_UK["date"], dataset_UK["new_deaths"], title = "New Deaths", color = dataset_UK["Variant"]) #hover_data = ["new_deaths", "population", "total_cases", "new_cases"])
fig.update_traces(mode = "markers + lines", hovertemplate = None)
fig.update_layout(hovermode = "x unified")
fig.show()

In [69]:
fig = px.line(dataset_UK,  dataset_UK["date"], dataset_UK["Fatality_Rate"], title = "Fatality Rate", color = dataset_UK["Variant"]) #hover_data = ["new_deaths", "population", "total_cases", "new_cases"])
fig.update_traces(mode = "markers + lines", hovertemplate = None)
fig.update_layout(hovermode = "x unified")
fig.show()

In [70]:
fig = px.line(dataset_UK,  dataset_UK["date"], dataset_UK["Fatality_Rate"], title = "Fatality Rate", color = dataset_UK["Lockdown"]) #hover_data = ["new_deaths", "population", "total_cases", "new_cases"])
fig.update_traces(mode = "markers + lines", hovertemplate = None)
fig.update_layout(hovermode = "x unified")
fig.show()

In [71]:
dataset_UK["Vaccine"].unique()

array(['No Vaccine', 'Astra Zeneca'], dtype=object)

In [72]:
labels = {
    "No Vaccine" : 0,
    "Astra Zeneca" : 1
}

In [73]:
dataset_UK["Vaccine"] = dataset_UK["Vaccine"].map(labels)

In [74]:
labels = {
    'First' : 0,
     'Alpha' : 1, 
     'Delta' : 2
}

In [75]:
dataset_UK["Variant"] = dataset_UK["Variant"].map(labels)

In [76]:
dataset_UK["Variant"].unique()

array([0, 1, 2])

In [77]:
dataset_UK.head()

,location,date,new_deaths_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,new_tests_per_thousand,total_vaccinations_per_hundred,gdp_per_capita,population,reproduction_rate,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred,new_cases,new_deaths,Mortality_Rate,Fatality_Rate,Lockdown,Variant,Vaccine
90078,United Kingdom,2020-05-01,10.311,69.676,32.083,216.878,1.092,0.0,39753.244,67886004.0,0.94,0.0,0.0,4730.0,700.0,0.147985,0.147992,0,0,0
90079,United Kingdom,2020-05-02,8.617,47.580,30.463,211.826,0.936,0.0,39753.244,67886004.0,0.90,0.0,0.0,3230.0,585.0,0.181106,0.181115,0,0,0
90080,United Kingdom,2020-05-03,3.727,43.853,29.594,208.997,1.005,0.0,39753.244,67886004.0,0.87,0.0,0.0,2977.0,253.0,0.084988,0.084985,0,0,0
90081,United Kingdom,2020-05-04,4.021,49.834,28.518,207.657,1.070,0.0,39753.244,67886004.0,0.82,0.0,0.0,3383.0,273.0,0.080688,0.080698,0,0,0
90082,United Kingdom,2020-05-05,10.680,54.341,27.605,202.148,0.986,0.0,39753.244,67886004.0,0.81,0.0,0.0,3689.0,725.0,0.196537,0.196530,0,0,0


In [78]:
dataset_UK.set_index("date", inplace = True)

In [79]:
dataset_UK.head()

,location,new_deaths_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,new_tests_per_thousand,total_vaccinations_per_hundred,gdp_per_capita,population,reproduction_rate,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred,new_cases,new_deaths,Mortality_Rate,Fatality_Rate,Lockdown,Variant,Vaccine
date,,,,,,,,,,,,,,,,,,,
2020-05-01,United Kingdom,10.311,69.676,32.083,216.878,1.092,0.0,39753.244,67886004.0,0.94,0.0,0.0,4730.0,700.0,0.147985,0.147992,0,0,0
2020-05-02,United Kingdom,8.617,47.580,30.463,211.826,0.936,0.0,39753.244,67886004.0,0.90,0.0,0.0,3230.0,585.0,0.181106,0.181115,0,0,0
2020-05-03,United Kingdom,3.727,43.853,29.594,208.997,1.005,0.0,39753.244,67886004.0,0.87,0.0,0.0,2977.0,253.0,0.084988,0.084985,0,0,0
2020-05-04,United Kingdom,4.021,49.834,28.518,207.657,1.070,0.0,39753.244,67886004.0,0.82,0.0,0.0,3383.0,273.0,0.080688,0.080698,0,0,0
2020-05-05,United Kingdom,10.680,54.341,27.605,202.148,0.986,0.0,39753.244,67886004.0,0.81,0.0,0.0,3689.0,725.0,0.196537,0.196530,0,0,0


In [80]:
features = ['location',
 'new_deaths_per_million',
 'new_cases_per_million',
 'icu_patients_per_million',
 'hosp_patients_per_million',
 'new_tests_per_thousand',
 'total_vaccinations_per_hundred',
 'gdp_per_capita',
 'population',
 'reproduction_rate',
 'people_fully_vaccinated_per_hundred',
 'people_vaccinated_per_hundred',
 'new_cases',
 'Mortality_Rate',
 'Fatality_Rate',
 'Lockdown', 'Variant', 'Vaccine',
 'new_deaths']

In [81]:
dataset_UK = dataset_UK[features]

In [82]:
dataset_UK.head()

,location,new_deaths_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,new_tests_per_thousand,total_vaccinations_per_hundred,gdp_per_capita,population,reproduction_rate,people_fully_vaccinated_per_hundred,people_vaccinated_per_hundred,new_cases,Mortality_Rate,Fatality_Rate,Lockdown,Variant,Vaccine,new_deaths
date,,,,,,,,,,,,,,,,,,,
2020-05-01,United Kingdom,10.311,69.676,32.083,216.878,1.092,0.0,39753.244,67886004.0,0.94,0.0,0.0,4730.0,0.147985,0.147992,0,0,0,700.0
2020-05-02,United Kingdom,8.617,47.580,30.463,211.826,0.936,0.0,39753.244,67886004.0,0.90,0.0,0.0,3230.0,0.181106,0.181115,0,0,0,585.0
2020-05-03,United Kingdom,3.727,43.853,29.594,208.997,1.005,0.0,39753.244,67886004.0,0.87,0.0,0.0,2977.0,0.084988,0.084985,0,0,0,253.0
2020-05-04,United Kingdom,4.021,49.834,28.518,207.657,1.070,0.0,39753.244,67886004.0,0.82,0.0,0.0,3383.0,0.080688,0.080698,0,0,0,273.0
2020-05-05,United Kingdom,10.680,54.341,27.605,202.148,0.986,0.0,39753.244,67886004.0,0.81,0.0,0.0,3689.0,0.196537,0.196530,0,0,0,725.0


In [83]:
dataset_UK_train = dataset_UK[ : 375]
dataset_UK_test = dataset_UK[ 375 : ]

In [84]:
len(dataset_UK_train)

375

In [85]:
len(dataset_UK_test)

30

In [86]:
X_train = dataset_UK_train.iloc[ : , : -1].drop("location", axis = 1)
X_test = dataset_UK_test.iloc[ : , : -1].drop("location", axis = 1)
Y_train = dataset_UK_train.iloc[ : , -1]
Y_test = dataset_UK_test.iloc[ : , -1]

In [87]:
regressor = RandomForestRegressor(random_state = SEED)
regressor.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [88]:
Y_pred = regressor.predict(X_test)

In [89]:
from sklearn.metrics import r2_score, mean_absolute_error
r2_score(Y_test, Y_pred)

0.9970858299947745

In [90]:
mean_absolute_error(Y_test, Y_pred)

0.1663333333333333

In [ ]:
ensemble = 